# SVM, Support Vector Machine

## 1. 기본

- SVM은 classfier이며 데이터를 잘 구분하는 hyperplane을 찾는 것이 목표다.
- hyperplane은 Yes-No를 가르는 기준, 즉 Classifier를 의미한다. P차원의 feature일 때 항상 P-1차원의 함수가 된다.
- 자주 쓰여지는 분류기이며 이유는 linear classifier이지만 kernel을 활용해서 non-linear 데이터도 분류할 수 있기 때문이다.

\begin{align}
\beta_0 + \beta_1X_1 + \beta_2X_2 + ... + \beta_pX_p = 0
\end{align}

## 2. Maximal Margin Classifier

### 2.1 개념

![svm](https://lukelushu.files.wordpress.com/2014/09/margin_width1.png)

- 데이터를 구분하는 classifier, 즉 hyperplane이 여러개 있을 수 있기 때문에 선택하는 기준이 필요하다.
- Margin을 최대화하는 hyperplane이 가장 훌륭한 classifier이고 그를 찾는 방법이다.
    + Margin: Support vector와 hyperplane의 거리(support vector와 hyperplane의 수선 거리)
    + Support vector: hyperplane과 가장 가까운 데이터들. 양쪽에 하나씩 있으니 최소 2개가 존재하게 된다.
- Margin을 최대화하는 이유는 generalization을 좋게하기 때문이다.(아래 2.2에서 더 이론적으로 설명)
    + generalization은 학습한 모델이 새로운 데이터에 대해서도 큰 편향 없이 잘 동작한다는 의미다. overfitting의 반대말이라 생각하면 되겠다.(underfitting은 학습 자체가 잘 안된 것)
    + 즉 Maximal margin classifier 학습 방식은 모델이 새로운 데이터에도 잘 적용될 수 있도록 학습할 때부터 데이터 편차에 융통성을 주는 방식이다. 여유 혹은 여지를 준다고 봐도 좋다.
- 위 설명에 따르면 직관적으로 다음 순서대로 구현해야할 것 같지만 계산이 힘들다. 다른 방식 활용할 것.
    + 하나의 hyperplane 지정
    + 각 데이터 간의 거리 측정
    + Support vector 찾기
    + Margin 계산
    + 해당 Margin이 다른 hyperplane의 값보다 큰지 비교
    
### 2.2 VC dimension

- dichotomy : 하나의 set을 두 개로 나눈다는 의미
    + $P1(x_1^1,x_2^1)$, $P2(x_1^2,x_2^2)$, $P3(x_1^3,x_2^3)$ 세 점이 있을 때 이를 한 set로 보고 두 개로 나누는 것
    + $[\emptyset, (A,B,C)]$ 로 나누든지, $[A, (B,C)]$로 나누든지 한다.
    + 세 점이 있으므로 총 8개의 subset이 나올 수 있다.
- shattering: 위 dichotomy를 통해 나오는 8개의 subset을 classifier가 다 표현할 수 있는지 판단
    + 점이 3개라면 하나의 선으로 8개의 subset을 모두 표현할 수 있다.
    + 점이 4개라면 16개의 subset을 하나의 선으로 표현 불가능(XOR 형태의 케이스는 불가)
- VC dimension(Vapnik Chervonenkis dimension)
    + classifier가 얼마나 복잡한 데이터를 분류할 수 있는지를 나타내는 metric
    + 정의: shattering할 수 있는 maximum number of data point
    + 예) 2차원에서 linear classifier의 VC dimension은 3(점 3개까지만 shattering 가능)
    + 일반화: d차원에서의 linear classifier의 VC dimension은 d+1이 됨
- VC dimension이 높으면 안좋다. VC dimension이 높은 classifier는 true error를 높인다. 안좋다라고 이해.
- SVM에서 마진을 최대화하는 것
    + $\rightarrow$ shattering 할 수 있는 가능성을 낮춘다.
    + $\rightarrow$ shattering 할 수 있는 potential dataset을 줄인다는 것
    + $\rightarrow$ 결국 VC dimension을 낮춘다는 것.
    + $\rightarrow$ True error를 낮춘다는 것

### 2.2 Margin 구하기

#### 2.2.1 수학적 접근으로 시작

**Constraint: Margin을 1로**

- hyperplane: $W^TX + b = 0$
- constraint: 하이퍼플레인 바깥에 있는 데이터 $x_n$에 대해서 $|W^Tx_n + b| = 1$ 임을 가정
    + 이 때 $x_n$은 하이퍼플레인과 가장 가까이 있는 데이터, 즉 서포트벡터다.
    + margin이 1이 아닐 수도 있는데 이렇게 가정하는 것은 하이퍼플레인을 기준으로 classification이 되는데 이 때 방향만이 중요하기 때문이다. 절대값을 씌워서 양의 방향이냐 음의 방향이냐만 알면 되고, 크기는 중요치 않다. 그래서 계산 편의를 위해 크기 1로 지정
- w 벡터는 하이퍼플레인과 orthogonal하다.
    + 하이퍼플레인 위의 두 점 $x_1, x_2$를 하이퍼플레인 수식에 집어넣고 빼면
    + $W^T(x_1 - x_2) = 0$ 꼴이 된다.
    + 두 벡터의 내적이 0이므로 직교한다고 볼 수 있다.
- margin 구하는 방법
    + $x_n$과 하이퍼플레인위의 점 $x_1$을 잇는 벡터 $(x_n - x_1)$를
    + 하이퍼플레인과 orthogonal한 벡터 w에 projection한 벡터의 길이가 margin이 된다.
    + 즉 벡터 $(x_n - x_1)$과 w를 내적한 값에 $||w||$을 나눠서 정규화해주면 계산 가능
- 위 수식을 풀어쓰면 맨 위에서 정의한 hyperplane과 constraint에 의해 아래 식으로 margin 구하는 식이 최종 정리된다.

$$
margin = {1 \over ||w||}
$$

#### 2.2.2 계산하기 쉽도록 변경

$$
y_n(W^Tx_n + b) \ge 1
$$

- **constraint** 수식 변경
    + 앞서 2.2.1에서 구한 margin 공식에서 margin을 최대화하려고 하면 constraint의 절대값 때문에 계산하기가 복잡하다. 절대값을 없애는 방법이 필요하다.
    + 위 수식처럼 Constraint를 절대값 없는 식으로 변형한다.
        - y를 1 또는 -1로 정하고 곱해서 항상 양수로 만들어준다.
        - 1보다 큰 이유는 2.2.1에서 constraint를 그렇게 정해놨기 때문이다.

$$
{1\over2}w^Tw
$$

- **Margin** 수식 변경: 식에 역수를 취해 minimize 최적화 방식으로 변형
    + 컨벤션이 그렇다. maximize보다는 minimize 형태의 최적화를 주로 사용한다.
    + 1/2를 곱해주는 이유는 $w^Tw$가 제곱의 형태라서 미분할 때 2가 내려온다. 수식을 간결하게 하기 위해 넣었다.

#### 2.2.3 Lagrange dual

- Lagrangian은 조건이 주어진 constrained problem에서 constraint들을 object function에 포함시켜서 보다 나은 형태로 풀기 위한 방법. 최대값 또는 최소값을 찾을 때 사용한다.
- 먼저 **slack**이란 constraint에서 우측의 1을 좌항으로 옮긴 식을 말한다.
    + $y_n(W^Tx_n + b) - 1$
- 2.2.2의 margin을 구하는 수식에서 모든 데이터에 대한 slack을 빼는 꼴이 Lagrange dual이다. 아래 수식과 같고 이를 최소화하면 된다.

$$
{1\over2}w^Tw - \sum_{n=1}^N \alpha_n(y_n(W^Tx_n + b) - 1)
$$

- alpha: 최적 solution에 각 데이터들이 미치는 영향이 다르기 때문에 알파(weight)를 곱해야한다.
- 라그랑지안을 계산할 때 $\alpha_n(y_n(w^Tx_n + b) - 1) = 0$ 이라는 제한조건이 존재한다.(아래에서 다시 설명할 것) 그렇게 되면 사실상 위의 식에서 두 번째 항은 0을 다 더해서 0이 되는데 그래도 의미없는 값은 아니다. 각 변수에 대해 미분을 해서 목표함수를 최적화하기 때문이다.
- 각 변수에 대해 최소, 최대화
    + `w`와 `b`에 대해선 값을 최소화해야한다 -> 위 식에서 양의 값이므로 줄여야 전체 식의 값이 줄어든다.
    + 각 slack에 대한 가중치인 $\alpha$는 최대화해야한다 -> 위 식에서 slack은 constraint에 의해 0이상인 값이고 알파를 곱해서 전체 식에서 빼는 것이기 때문에 알파가 커질수록 전체 식의 값이 작아질 수 있기 때문이다.
- w와 b에 대해 미분하고 0의 값을 줘서 최소화 지점을 구할 수 있다.
    + $dwL = w - \sum_{n=1}^N{\alpha_ny_nx_n} = 0$
    + $dbL = -\sum_{n=1}^N{\alpha_ny_n} = 0$
- 위 미분해서 구한 두 식을 Lagrange dual 꼴에 대입을 한다. 첫 번째 식은 w에 대해서 정리하고, 둘 째 식은 $\alpha_ny_n$의 sum이 0인 것이므로 라그랑지 듀얼 수식에 대입하여 정리하면 아래 식처럼 w 없이 y, x, alpha 값만 남길 수 있다. 

$$
L = \sum_{n=1}^N \alpha_n - {1\over2}\sum_{n=1}^N\sum_{m=1}^N y_ny_m\alpha_n\alpha_mx_nx_m
$$

- 이 때 x와 y는 데이터에서 가져오면 되는 이미 "알고 있는" 값이고 사실상 알파값만 구하면 된다.
- 위 식에서 $y_ny_m, x_nx_m$이 linear term이고, $\alpha_n\alpha_m$은 quadratic term이다. 이 상황에서 알파 값을 quadratic programming 기법으로 쉽게 구할 수 있다.
    + 다만 조건은 위에서 구한 모든 데이터에 대한 알파와 y의 곱의 합은 0이 되어야한다는 것과
    + 알파는 0이상의 값이라는 것(최적화 기법이 그러하다)
- 위 조건과 알파에 대해 maximize하는 방향으로 Quadratic programming solver를 돌리면 대부분의 알파 값이 0이 나온다.
    + $\alpha_n(y_n(w^Tx_n + b) - 1) = 0$ 알파와 slack을 곱한 값이 0이라는 제한조건이 있다.
    + 서포트 벡터라면 슬랙이 0이기 때문에 알파가 양의 값을 가질 수 있고,
    + 하이퍼플레인과 더 떨어져있는 다른 데이터들은 slack이 0 이상이라서 알파가 0이 될 수 밖에 없다.
- 즉 슬랙이 0이고, 알파가 0이 아닌 데이터들을 서포트벡터라 하며, 하이퍼플레인이 정해지기 위해선 최소 3개(한 쪽에 2개, 다른 한 쪽에 1개)의 서포트벡터가 필요하다.
- SVM의 모델이 한 번 만들어지면 최소 3개 이상의 서포트벡터를 알게되는 것이고, 새로운 데이터가 들어오면 단순히 이 서포트 벡터 중 하나와 내적해서 "방향"만 알면 classify가 가능해진다. 그래서 계산 비용이 적다.

### 2.3 Weight vector의 크기를 1로 하는 방법

$$
\text{distance} = {|\ ax + by + c\ | \over \sqrt{a^2 + b^2}}
$$

- 위의 점과 선의 거리 공식을 이용할 것이다. 이를 활용해 support vector와 hyperplane 간의 거리인 margin을 구한다.
- 먼저 위 분자의 $ax + by + c$ 에 절대값을 씌워야한다.
    + $ax + by + c$ 는 feature 2개가 있는 데이터 평면을 가정했을 때 hyperplane인 $\beta_0 + \beta_1x_1 + \beta_2x_2$과 같다.
    + 이 hyperplane의 결과값이 0인 선을 기준으로 데이터가 분류되게 된다. 즉 양수면 A, 음수면 B이다.
    + 계산의 편의를 위해 y 값을 1 또는 -1로 설정한다. logistic regression에서 y값을 0 또는 1로 설정했던 것과 같은 맥락이다.
    + $y_i(\beta_0 + \beta_1x_1 + \beta_2x_2) > 0$ 처럼 hyperplane에 y 값을 곱하면 항상 0보다 크게 된다. 즉 절대값을 씌운 것과 같은 맥락이다.
- 이 때 $a^2 + b^2$ 값이 1이라면 거리는 분모가 사라져서 margin은 $y_i(\beta_0 + \beta_1x_1 + \beta_2x_2)$ 공식의 값이된다.

> 위처럼 $\beta_1,\ \beta_2$의 값이 작게 조정되면 $\beta_0$의 값이 과하게 커지거나 작아질 수 있다. 하지만 $\beta_0$의 값은 데이터에 따라서 설정되는 것이기 때문에 나쁜 문제는 아니다.

### 2.4 결론

- 첫 번째 방식: $|W^Tx_n + b| = 1$ 조건 하에서 margin은 ${1 \over ||w||}$ 으로 표현

$$
argmax_{\beta_0,\beta_1,\beta_2} M \\
\text{subject to }|W^Tx_n + b| = 1 \\
margin = {1 \over ||w||}
$$

- 두 번째 방식: $\beta_1^2 + \beta_1^2 = 1$ 조건 하에서 margin은 $y_i(\beta_0 + \beta_1x_1 + \beta_2x_2)$ 로 표현

$$
argmax_{\beta_0,\beta_1,\beta_2} M \\
\text{subject to }\beta_1^2 + \beta_2^2 = 1 \\
y_i(\beta_0 + \beta_1X_1 + \beta_2X_2) \ge M
$$

되며 이 결과값, 즉 margin을 최대화하는 $\beta$들의 값을 찾아내는 것이 목표다. 그리고 이 최적화 문제는 **Quadratic program** 방식으로 푼다.

## 3. Soft Margin Classifier

- Maximal Margin Classifier에서 발생하는 문제를 해결하기 위해 등장했다.
- 문제
    + 어떻게 선을 그어도 데이터를 완벽하게 구분하는 hyperplane이 없는 경우
    + Outlier가 있어서 hyperplane이 overfitting 되는 경우, 즉 generalization이 좋지 않은 경우
- 해결: 학습할 때 Margin에 여유(여지)를 두자
    + Margin 안에 다른 데이터가 있어도 오케이. 즉 support vector라고 정한 데이터보다 더 hyperplane에 가까운 데이터가 존재해도 괜찮다.
    + Hyperplane 반대 쪽에 다른 데이터가 있어도 괜찮다.

$$
argmax_{\beta_0,\beta_1,\beta_2,\epsilon_1,\epsilon_2,\epsilon_3} M \\
\text{subject to }\beta_1^2 + \beta_2^2 = 1 \\
y_i(\beta_0 + \beta_1X_1 + \beta_2X_2) \ge M(1 - \epsilon_i) \\
\epsilon \ge 0 \\
\sum_{i=1}^N\epsilon_i \le C
$$

- 수식으로 표현하면 위와 같다. 즉 margin의 일정 비율보다 더 크면 오케이라는 것. 여지를 둔 것이다.
- epsilon들을 slack variable이라 하고, C를 Penalty parameter라고 한다.
- C가 커질수록 관대하게 margin을 설정하겠다라는 의미다.
    + `C = 0` : Maximal margin classifier와 같다. 여유 없이 딱 Margin을 계산하겠다는 것.
- 역시 Quadratic program 방식으로 최적화 해를 구한다.

## 4. Support Vector Machine

### 4.1 Mapping function

![mapping function](http://omega.albany.edu:8008/machine-learning-dir/notes-dir/ker1/phiplot.gif)

- 데이터를 선형으로 구분하지 못한다면 Soft margin classifier으로도 분류를 제대로 할 수 없다.
- 데이터를 좀 더 쉬운 모양으로 바꿀 필요가 있다. 위 이미지처럼 데이터를 더 쉽게 설명할 수 있도록 다른 차원으로 데이터를 이동시켜야한다.
- mapping function $\phi$를 이용한다. (아래 mapping function은 타원을 의미한다.)

$$
\phi(X_1,X_2) \rightarrow (Z_1,Z_2,Z_3) \equiv (X_1^2,\sqrt2X_1X_2,X_2^2)
$$

- 다만 mapping function으로 차원을 높이면 hyperplane을 찾아서 분류할 수 있지만 계산 비용이 높다. 그래서 **Kernel** 함수를 쓴다.

### 4.2 Kernel trick

- Kernel 함수란 새로운 공간에서 두 데이터의 similarity를 측정하는 함수이고 내적(inner product)를 활용한다.
- mapping function을 적용하는 과정을 생략하기 때문에 계산 비용을 낮출 수 있다.(mapping function의 의미는 내재하고 있다)

$$
K(x^{(1)}, x^{(2)}) \equiv (<x^{(1)}, x^{(2)}>)^2 = (x_1^{(1)},x_1^{(2)} + x_2^{(1)},x_2^{(2)})^2 \\
= <\phi(x^{(1)}), \phi(x^{(2)})>
$$

- 위 공식대로 Kernel 함수의 정의는 x^(1)과 x^(2)의 내적에 제곱을 한 것이다.
- 그런데 이것은 mapping function을 적용한 X1, X2 벡터를 내적을 한 값과 같다.
- 그렇기 때문에 mapping function을 굳이 적용해서 차원을 올리지 않고, 기존 차원에서 바로 Kernel 함수를 사용하면 벡터 간 내적 연산이 가능하다.
- 다음은 Kernel 함수의 종류이고 최적화하는 방법은 다양하다.
    + Linear kernel
    + Quadratic kernel
    + Polynomial kernel of degree d
    + RBF(Radial Basis Function) of Gaussian kernel
- SVM에서 가장 자주 쓰이는 커널 함수 4가지다.

$$
\text{linear :} <\vec{x},\vec{y}> \\
\text{Polynomial :} (\gamma <\vec{x},\vec{y}> + r)^d \\
\text{Radial basis function (rbf) :} \exp(-\gamma |\vec{x}-\vec{y}|^2) \\
\text{Sigmoid :} \tanh(\gamma <\vec{x},\vec{y}> + r) \\
\text{where } \gamma > 0, d \in \mathbb{Z},  r \in \mathbb{R}
$$

### 4.3 KMOOC 오혜연 교수님 설명 정리

#### 4.3.1 kernel trick을 쓰는 이유

$$
L = \sum_{n=1}^N \alpha_n - {1\over2}\sum_{n=1}^N\sum_{m=1}^N y_ny_m\alpha_n\alpha_mk(x_nx_m)
$$

- SVM은 linear classification 방법이다. 데이터에 따라 리니어로 구분되지 않을 수 있기 때문에 차원을 올린다.
- 차원을 올리면 계산 비용이 높아지는 건 맞다. 위 Lagrange dual 공식에서 문제가 되는 부분은 $k(x_nx_m)$ 부분이다. 앞쪽의 나머지 공식들은 dimension이 어떻든 변하지 않는 부분이다. k는 커널 함수를 의미하고 고차원으로 올렸음을 의미한다. 고차원 벡터라는 의미로 $z_nz_m$ 이라고 대치해서 나타내기도 한다.
- 즉 $k(x_nx_m)$ 내적 계산만 잘 할 수 있으면 나머지 것은 문제 없다.
- 여기서 k, kernel 함수가 여러 종류가 있다. 가우시안 커널도 있고, Radial basis function 커널도 있고, 폴리노미얼 커널도 있고 다양하다. 앞서 설명한대로 SVM은 linear classifier이고 제대로 classify하려면 차원을 무한대로 높일 수도 있다. 그런데 결국 차원을 무한대로 높였을 때 해야할 일은 그 시점의 "내적"을 구하는 것이다. 즉 커널의 역할은 굳이 그렇게 높이 차원을 올리지 않더라도 저차원에서 그 때의 내적 값을 구할 수 있도록 해준다. 이것이 kernel trick이다.

#### 4.3.2 kernel trick 예시

$$
\text{Gaussian kernel} = exp(-{||x_n - x_m||^2 \over 2\sigma^2})
$$

- 예를 들어 가우시안이라면, 두 데이터가 있을 때 가우시안으로 similarity를 구하는 것. 즉 한 데이터 포인트를 X축의 0점으로 하는 정규분포 곡선을 가상으로 그린 후 다른 데이터 포인트의 값을 계산하는 것이다. 가까이 있으면 값이 1에 가깝게 높게 나올 것이고, 멀리 있으면 낮을 것.
- 즉 $x_n - x_m$ 값은
    + 데이터가 서로 가까이 있으면 1에 가까운 값이 나와서 가우시안 텀이 0에 가까워지고,
    + 멀리 있으면 값이 0에 가까워지기 때문에 가우시안 텀이 1에 가까워진다.
- 그래서 차원이 infinite이더라도 쉽게 값을 계산할 수 있어서 좋다.
